In [3]:
from datetime import datetime
import polars as pl
import os
import sys
import plotly.express as px
import polars.selectors as cs


PROJECT_PATH = os.path.abspath(os.path.join("..", ".."))
PIPELINE_PATH = os.path.join(PROJECT_PATH, "pipeline")
DATA_DIRECTORY = os.path.join(PROJECT_PATH, "data")

if PIPELINE_PATH not in sys.path:
    sys.path.append(PIPELINE_PATH)
    
from utils.paths import PROCESSED_PICARRO_DATA_DIRECTORY
from utils.calibration_processing import process_bottle

assert(os.path.exists(PROCESSED_PICARRO_DATA_DIRECTORY))

In [8]:
start_date = datetime(2023, 12, 19, 0, 0, 0)
end_date = datetime(2023, 12 , 20, 0, 0, 0)


# Load Picarro Data
# DWD Picarro: "Calibrated_1_min_DWD_Picarro_G2301_413.parquet"
df_p = pl.scan_parquet(os.path.join(PROCESSED_PICARRO_DATA_DIRECTORY,"Calibrated_1_min_DWD_Picarro_G2301_413.parquet"))
df_p = df_p.filter(pl.col("datetime").is_between(start_date, end_date)) \
    .select(["datetime", "picarro_corrected", "h2o_reported"]) \
    .collect()
    
df_p.head(1).vstack(df_p.tail(1))

datetime,picarro_corrected,h2o_reported
datetime[ms],f64,f64
2023-12-19 00:00:00,519.542796,0.874734
2023-12-20 00:00:00,440.629898,0.64298


In [9]:
def plot_sensor_measurement(df, col_name: str, filter = None):
    if filter != None:
        df = df.groupby_dynamic("datetime", every=filter).agg(cs.numeric().mean())
        
    fig = px.line(
        df,
        x="datetime",
        y=col_name,
        markers=True,
        title=col_name,
    )
    fig.show()
    
def process_measurement(start_date, end_date, plot=False):
    df_p_bottle = df_p.filter(pl.col("datetime").is_between(start_date, end_date))

    data = df_p_bottle.select(pl.col("picarro_corrected")).to_series().to_list()
    data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

    median = process_bottle(data_cut, ignore_len=True) 

    print(f"Picarro Median Reading: {median}")

    # Plots
    if plot:
        plot_sensor_measurement(df_p_bottle, col_name="picarro_corrected")
        plot_sensor_measurement(df_p_bottle, col_name="h2o_reported")

# Bottles on 19.12.2023

In [24]:
# bottle 36

# PICARRO
start_date = datetime(2023, 12, 19, 10, 39, 0)
end_date = datetime(2023, 12, 19, 10, 51, 0)

process_measurement(start_date, end_date, plot = True)

Picarro Median Reading: 400.800778870219


In [11]:
# bottle 38

# PICARRO
start_date = datetime(2023, 12, 19, 11, 32, 0)
end_date = datetime(2023, 12, 19, 11, 48, 0)

process_measurement(start_date, end_date)


Picarro Median Reading: 814.6821277460296


In [12]:
# bottle 21		12:58	13:09	397,7

# PICARRO
start_date = datetime(2023, 12, 19, 11, 53, 0)
end_date = datetime(2023, 12, 19, 12, 3, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 400.30566345307204


In [13]:
# bottle 3		13:13	13:23	388.8

# PICARRO
start_date = datetime(2023, 12, 19, 12, 8, 0)
end_date = datetime(2023, 12, 19, 12, 16, 0)

process_measurement(start_date, end_date)


Picarro Median Reading: 391.3033807620875


In [14]:
# bottle 12		13:25	13:35	811,1

# PICARRO
start_date = datetime(2023, 12, 19, 12, 20, 0)
end_date = datetime(2023, 12, 19, 12, 29, 0)

process_measurement(start_date, end_date)


Picarro Median Reading: 816.2064719528489


In [15]:
# bottle 18		13:36	13:45	398,1

# PICARRO
start_date = datetime(2023, 12, 19, 12, 32, 0)
end_date = datetime(2023, 12, 19, 12, 39, 0)

process_measurement(start_date, end_date)


Picarro Median Reading: 400.58547710534026


In [16]:
# bottle 1		13:49	13:59	793,2

# PICARRO
start_date = datetime(2023, 12, 19, 12, 42, 0)
end_date = datetime(2023, 12, 19, 12, 52, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 798.1999007481917


In [17]:
# bottle 28		14:01	14:10	806.0

# PICARRO
start_date = datetime(2023, 12, 19, 12, 55, 0)
end_date = datetime(2023, 12, 19, 13, 4, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 811.1014562645336


In [18]:
# bottle 33		14:11	14:21	800,7

# PICARRO
start_date = datetime(2023, 12, 19, 13, 6, 0)
end_date = datetime(2023, 12, 19, 13, 14, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 805.6766812434226


In [19]:
# bottle 15		14:22	14:32	802,7

# PICARRO
start_date = datetime(2023, 12, 19, 13, 18, 0)
end_date = datetime(2023, 12, 19, 13, 26, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 807.8693381332425


In [20]:
# bottle 24		15:09	15:20	801,8

# PICARRO
start_date = datetime(2023, 12, 19, 14, 3, 0)
end_date = datetime(2023, 12, 19, 14, 13, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 806.81842457468


In [21]:
# bottle 5		15:22	15:32	404,4

# PICARRO
start_date = datetime(2023, 12, 19, 14, 17, 0)
end_date = datetime(2023, 12, 19, 14, 26, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 407.0568730904048


In [22]:
# bottle 8		15:33	15:43	410,7

# PICARRO
start_date = datetime(2023, 12, 19, 14, 28, 0)
end_date = datetime(2023, 12, 19, 14, 37, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 413.3563371658821


In [23]:
# bottle 23		15:44	15:54	388.0

# PICARRO
start_date = datetime(2023, 12, 19, 14, 39, 0)
end_date = datetime(2023, 12, 19, 14, 47, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 390.48890935275824
